#Importamos librerias

In [ ]:
import pandas as pd
#!pip install pandas_summary
#from pandas_summary import DataFrameSummary
import numpy as np

from matplotlib import pyplot as plt
import pickle #para guardado de datos
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Input,Add
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling2D,RandomFlip,RandomRotation
from keras.layers.core import Dense, Flatten, Activation

In [ ]:
import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU no encontrada')
# print('Encontrada GPU: {}'.format(device_name))

# Funciones de carga y guardado de datos 

In [ ]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
def pickle_save_as_csv(lista, nombre_lista):
  # Se transforman los archivos de bytes a strings normales
  label_names = [s.decode("utf-8") for s in lista]
  pd_label_names = pd.DataFrame({'Classes' : label_names})
  pd_label_names['Labels'] = pd_label_names.index
  # Se guarda todo como un csv:
  pd_label_names.to_csv(f'/content/drive/Shareddrives/Redes neuronales/TP3/Dataset Pre-Procesado/{nombre_lista}')
  return pd_label_names


#Funciones Data Augmentation

In [ ]:
def data_augmentation(x_train_data,y_train_data,random_flip=True,mode_flip ="horizontal_and_vertical", random_rotation=True,factor_rotation = 0.2,random_crop=False,height_crop=10,width_crop=10,random_translation=False,height_translation=10,width_translation=10,random_zoom=False,height_zoom=10,width_zoom=10,random_contrast=False,factor_contrast=0.2):
  data_augmentation = tf.keras.Sequential()
  if random_flip: data_augmentation.add(layers.RandomFlip(mode_flip))
  if random_rotation: data_augmentation.add(layers.RandomRotation(factor_rotation))    
  #if random_crop: data_augmentation.add(layers.RandomCrop(height_crop,width_crop)) #crops image
  if random_translation: data_augmentation.add(layers.RandomTranslation(height_translation, width_translation)) 
  if random_zoom: data_augmentation.add(layers.RandomZoom(height_zoom, width_zoom))
  if random_contrast: data_augmentation.add(layers.RandomContrast(factor_contrast))
  x_train_augmented = np.append(x_train_data,data_augmentation(x_train_data),axis=0)
  y_train_augmented = np.append(y_train_data,y_train_data,axis=0)

  return x_train_augmented,y_train_augmented

# Funciones CNN Alexnet

In [ ]:
def CNN_AlexNet(n_filters=20,loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy']):
  model = Sequential()

  model.add(Conv2D(filters=n_filters, kernel_size=(11, 11), input_shape=(32,32,3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(filters=n_filters, kernel_size=(5, 5), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())


  model.add(Conv2D(filters=n_filters, kernel_size=(3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Flatten())

  model.add(Dense(units=500, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(units=200, activation='relu')) #84
  model.add(Dropout(0.5))
  model.add(Dense(units=100, activation = 'softmax')) #10

  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  model.summary()
  return model

# Función CNN tipo VGG-11

In [ ]:
def CNN_VGG11(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy']):
  model = Sequential()

  model.add(Conv2D(filters=16, kernel_size=(3, 3), input_shape=(32,32,3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
  model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
  model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D())

  model.add(Flatten())

  model.add(Dense(units=256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(units=256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(units=100, activation = 'softmax'))

  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  model.summary()

  return model

# Función ResNet50

In [ ]:
net_input = Input(shape=(32,32,3))

def res_block_first(input_layer,filters=16):
    
    #Primera capa
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(input_layer)
    aux_net=BatchNormalization()(net)
    
    #conexión directa
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(aux_net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    
    #Shortcut
    net=Add()([aux_net,net])
    
    net=Activation("relu")(net)
    net=MaxPooling2D()(net)
    
    return net

def res_block(input_layer,filters=16):
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(input_layer)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    
    #Shortcut
    net=Add()([input_layer,net])
    net=Activation("relu")(net)
    
    return net

def res_block_dec(input_layer,filters=32):
    
    input_layer=MaxPooling2D()(input_layer)
    net=Conv2D(filters=filters, kernel_size=(1, 1), padding='same')(input_layer)
    aux_net=BatchNormalization()(net)
    
    #conexión directa
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(input_layer)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Activation("relu")(net)
    
    net=Conv2D(filters=filters, kernel_size=(3, 3), padding='same')(net)
    net=BatchNormalization()(net)
    net=Add()([aux_net,net])
    net=Activation("relu")(net)
    
    return net

net=res_block_first(net_input,filters=16)
net=res_block(net,filters=16)
net=res_block(net,filters=16)
net=res_block(net,filters=16)
net=res_block_dec(net,filters=32)
net=res_block(net,filters=32)
net=res_block(net,filters=32)
net=res_block(net,filters=32)
net=res_block_dec(net,filters=64)
net=res_block(net,filters=64)
net=res_block(net,filters=64)
net=res_block(net,filters=64)
net=res_block_dec(net,filters=128)
net=res_block(net,filters=128)
net=res_block(net,filters=128)
net=res_block(net,filters=128)

net = AveragePooling2D() (net)

net=Flatten()(net)

net=Dense(units=256, activation='relu')(net)
net=Dropout(0.1)(net)
net=Dense(units=256, activation='relu')(net)
net=Dropout(0.1)(net)
net=Dense(units=10, activation = 'softmax')(net)

model=Model(net_input,net)

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 16)   2320        ['batch_normalization[0][0]']

#Función para entrenar

In [ ]:
def train(model,monitor,filepath,epocs,batch_size,x_train,y_train_categorical,x_valid,y_val_categorical,verbose,loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy']):

  earlystop = EarlyStopping(monitor=monitor, min_delta=0, patience=5, verbose=verbose)
  checkpointer = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True) 

  history = model.fit(x_train, 
          y_train_categorical,
          epochs=epocs, batch_size=batch_size, 
          verbose=verbose, 
          validation_data = (x_valid, y_val_categorical),
          callbacks=[checkpointer,earlystop] , workers=-1 #, earlystop
      )
  return history,model

# Funciones de creación de modelo

In [ ]:
def crear_automatic_MLP(shape, cant_Neuronas, loss_function, optimizer_chosen, principal_metrics, set_weights = "imagenet", layer_trainable = True):
  # Se crea el modelo
  model_resnet = ResNet50(alpha=1.0, include_top = True, weights= set_weights, input_shape = shape, classes=100)

  if layer_trainable == False:
    # Si layer_trainable es False, quiere decir que al entrenar el modelo, los pesos no cambian.
    for layer in model_resnet.layers:
      layer.trainable = False
      print('Cero pesos entrenables')

  print("Capas convolucionales del modelo:")
  model_resnet.summary()

  # Se crean e inicializan los Layers del modelo
  input_layer = model_resnet.layers[0].input
  output_resnet = model_resnet.layers[-1].output

  x = GlobalAveragePooling2D()(output_resnet)
  x = Dense(cant_Neuronas, activation = "softmax")(x)

  # Se crea el modelo completo
  model = Model(inputs = input_layer, outputs = x)
  print('Modelo completo:')
  model.summary()

  # Se compila el modelo
  model.compile(loss = loss_function, optimizer = optimizer_chosen, metrics = principal_metrics)

  return model


In [ ]:
def crear_manual_MLP(shape, filter_conv, kernel_size, activation_func, scalar_filter ,pool_size, classes):
  model = Sequential()
  capa = 0 
  # Se crea la base convolucional:
  while len(activation_func) > capa:
    model.add(layers.Conv2D(scalar_filter[capa]*filter_conv, kernel_size, activation=activation_func[capa], input_shape= shape))
    model.add(layers.MaxPooling2D(pool_size))
    capa +=1

  print("Arquitectura del modelo hasta ahora:")
  model.summary()

  # Se agregan capas densas:
  model.add(layers.Flatten())
  model.add(layers.Dense(scalar_filter[capa-1]*filter_conv, activation='relu'))
  model.add(layers.Dense(classes))

  print("Arquitectura final del modelo:")
  model.summary()

  return model

# Funciones de entrenamiento modelo

In [ ]:
def entrenar_MLP(model, X_train, Y_train, porcentaje_val_split, cant_epochs, batch_size, verbose, path_model):
  checkpoint = ModelCheckpoint(path_model, monitor='val_Accuracy', verbose=verbose, save_best_only=True)
  history = model.fit(X_train, Y_train.flatten(), validation_split = porcentaje_val_split, epochs=cant_epochs, batch_size= batch_size, callbacks=[checkpoint] , verbose=verbose, workers=-1)
  return history, model

In [ ]:
def entrenar_manual_MLP(model, X_train, Y_train, porcentaje_val_split, cant_epochs, verbose, loss_function, optimizer_chosen, principal_metrics):
   # Se compila el modelo
  model.compile(loss = loss_function, optimizer = optimizer_chosen, metrics = principal_metrics)

  # Se entrena el modelo
  history = model.fit(X_train, Y_train.flatten(), validation_split = porcentaje_val_split, epochs=cant_epochs, verbose = verbose)

  return history,model

# Función de visualización

In [ ]:
def visualize_data(images, categories, class_names):
    fig = plt.figure(figsize=(14, 6))
    fig.patch.set_facecolor('white')
    for i in range(3 * 7):
        plt.subplot(3, 7, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(images[i])
        class_index = categories[i].argmax()
        plt.xlabel(class_names[class_index])
    plt.show()

# Predicción de clases y submission

In [ ]:
def predic(test,path_results,model,path_model):
  model.load_weights(path_model)

  y_test_pred = model.predict(test)
  y_test_pred = y_test_pred.argmax(axis=1)

  #submission
  df=pd.DataFrame(data=y_test_pred, columns=["label"])
  df.index.name = "Id"
  df.to_csv(path_results)

  return df